<center># Web application development </center>

In this project we will develop and deploy a web application to a cloud service making it accessible to the public. The projects tasks include opening accounts on github and render.com. Git repositary with README.md and gitignore files will be created. Jupyter notebook platform, VS Code (as an IDE) and python with its packages necessary to perform the project taks will be used.  
We will work on the dataset on car sales advertisements. First, using Jupyter notebook in VS Code IDE the dataset will be preprocced (working on missing and duplicated values). Further analisys will be done by excluding outlires and finding the correlations between column values. The revealed correlations we will visualized by plotting (scatter and histogram) the figures. These results will be stored in EDA.ipynb.
Next, in order to develop web applications we create app.py file in the root directory of the project. The main results including the obtained figures in EDA.ippynb will be copied to app.py. After getting all files with necessary results and contents we will commit and push them to the git repository. Committing and pushing processes of any update/change on the git repository files and directories will be done repeatedly until getting the final results on the web app.  

Below we install necessary packages

In [ ]:
%pip install pandas 
%pip install altair 
%pip install ipykernel 
%pip install streamlit 
%pip install plotly
%pip install --upgrade nbformat

importing packages

In [54]:
import pandas as pd
import streamlit
import altair as alt
import plotly.express as px

loading the data file

In [55]:
vehicles = pd.read_csv('~/Notebook/Project_4/vehicles_us.csv')

EDA of the data

In [56]:
vehicles.info()
vehicles.isna().sum()  # no wholly missing rows
vehicles.duplicated().sum()  # no duplicated rows
vehicles.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


EDA of 'model_year' columns

In [57]:
vehicles['model_year'].isna().sum()    # it shows 3619 missing values
vehicles['model_year'] = vehicles.groupby('model')['model_year'].transform(lambda x: x.fillna(x.median()))
vehicles['model_year'].isna().sum()     # missing values were filled with ['model_year'].median(). ['model_year'] has no more missing values. 

0

EDA of 'cylinders' columns

In [58]:
vehicles['cylinders'].isna().sum()    # it shows 5260 missing values
vehicles['cylinders'] = vehicles.groupby('model')['cylinders'].transform(lambda x: x.fillna(x.median()))
vehicles['cylinders'].isna().sum()     # missing values were filled with ['cylinders'].median(). ['cylinders'] has no more missing values. 

0

EDA of 'odometer' columns

In [59]:
vehicles['odometer'].isna().sum()    # it shows 5260 missing values
vehicles['odometer'] = vehicles.groupby('model_year')['odometer'].transform(lambda x: x.fillna(x.median()))
vehicles['odometer'].isna().sum()     # missing values were filled with ['cylinders'].median(). ['cylinders'] has no more missing values. 
vehicles['odometer'].isna().sum()     # only one missing value remaining. 
missing_odometer_rows = vehicles[vehicles['odometer'].isna()]
missing_odometer_rows       # this is the only car produced in 1929, and its ododmeter NaN, so its median=NAN, for our data analysis it can be considered as an outlier. 

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
45694,18000,1929.0,ford f-150,good,8.0,gas,NaN,manual,other,silver,NaN,2018-11-18,59


'model_year' is strongly right-skewed, and consider only cars produced since 1995, excluding older cars as outlires 

In [60]:
lower_year = 1995
vehicles = vehicles[vehicles['model_year'] >= lower_year]
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50698 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         50698 non-null  int64  
 1   model_year    50698 non-null  float64
 2   model         50698 non-null  object 
 3   condition     50698 non-null  object 
 4   cylinders     50698 non-null  float64
 5   fuel          50698 non-null  object 
 6   odometer      50698 non-null  float64
 7   transmission  50698 non-null  object 
 8   type          50698 non-null  object 
 9   paint_color   41583 non-null  object 
 10  is_4wd        25297 non-null  float64
 11  date_posted   50698 non-null  object 
 12  days_listed   50698 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.4+ MB


Here, we study the dependence of the car prices on their model years

In [61]:
model_year_order = vehicles.groupby('model_year')['price'].agg(['mean', 'count']).reset_index()
model_year_order.tail(20)

fig_model_year = px.scatter(model_year_order, x='model_year', y='mean', title='model year vs price')
fig_model_year.update_layout(xaxis_title='Model year', yaxis_title='Average price')
fig_model_year.show()

The above shown figure demonstrates the gradual increase of avarage prices of cars from older to newer ones. 

Next, we investigate which model year cars are most popular. 

In [63]:
hist_model_year = px.bar(model_year_order, x='model_year', y='count', 
             labels={'model_year': 'Model Year', 'count': 'Frequency'}, title='Histogram of Car Model Years Frequencies')
hist_model_year.update_traces(marker=dict(line=dict(width=2, color='Blue')))
hist_model_year.show()

The above shown histogram demonstrates that the cars produced 2011 - 2013 are most popular ones. The decrease corresponding to 2009 - 2010 can be caused by the crisis in economy. 

Here, we will investigate the dependence of the average price of cars on their conditions.

In [64]:
 condition_price = vehicles.groupby('condition')['price'].agg(['mean', 'count']).reset_index()
condition_order = ['new', 'like new', 'excellent', 'good',  'fair', 'salvage', ]
condition_price['condition'] = pd.Categorical(condition_price['condition'], categories=condition_order, ordered=True)
condition_sorted = condition_price.sort_values('condition')
condition_sorted.head(13)

fig_condition = px.scatter(condition_sorted, x='condition', y='mean', title='condition vs price')
fig_condition.update_layout(xaxis_title='Car condition', yaxis_title='Average price')

fig_condition.show()

This figure shows the gradual decrease of the average price of cars toward older cars. 

Here, we study what car conditions are most popular

In [65]:
hist_condition = px.bar(condition_sorted, x='condition', y='count', 
             labels={'condition': 'Condition', 'count': 'Frequency'}, title='Histogram of Car Condition Frequencies')
hist_condition.update_traces(marker=dict(line=dict(width=2, color='Blue')))
hist_condition.show()

This histogram shows that the used cars in excellent condition are most popular. 